# k-Nearest Neighbors #

A machine learning algorithm that classifies a data point using the labels of the $k$ nearest data points (neighbors). This is commonly handled with majority vote.

Generally Euclidean distance can be used as the "nearness" measure, though for discrete variables other metrics like Hamming distance are used.

The Euclidean distance between two data points $X$ and $Y$ each with $n$ features is:

<center>$d(X, Y) = \sqrt{(x_1 - y_1)^2 + \dotsc + (x_n - y_n)^2}$</center>

This can also be extended to a weighted classification system by weighing the vote of each neighbor by its nearness to the data point. From this standpoint, kNN is a weighting of 1/k to all k nearest neighbors and weight of 0 for the other data points.

In [21]:
# import packages

from sklearn import datasets, model_selection, metrics
from sklearn.neighbors import KNeighborsClassifier as knn

### Import the Data ###

Import built-in iris data set and split it into a training and test set. Since there is some parameter tuning needed for setting $k$, I'll also create a validation set.

The validation set will be created to have 20% of the data points from the training set.

In [5]:
data = datasets.load_iris()
X = data['data']
Y = data['target']

X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X, Y, train_size=0.7)
X_train,X_val,Y_train,Y_val = model_selection.train_test_split(X_train, Y_train, train_size=0.8)

### Construct the Model ###

Test different values for $k$ to find the "optimal" number of neighbors to use for classification. The scikit-learn implementation defaults the number of neighbors to 5 and odd numbers can be useful for breaking ties in the majority voting implementation. In the training set there are 84 data points.

Therefore, for illustrative purposes and analysis I will test all odd numbers from 3 to 85. The latter of which is the same as using essentially all the data points for classification.

In [19]:
k_vals = [i for i in range(3, 85, 2)]

for k in k_vals:
    model = knn(n_neighbors=k, weights='uniform')
    model.fit(X_train,Y_train)
    acc = model.score(X_val, Y_val)
    print(k, ':', acc)

3 : 0.9523809523809523
5 : 0.9523809523809523
7 : 0.9523809523809523
9 : 0.9523809523809523
11 : 0.9523809523809523
13 : 0.9523809523809523
15 : 0.9523809523809523
17 : 0.9523809523809523
19 : 0.9523809523809523
21 : 0.9523809523809523
23 : 1.0
25 : 1.0
27 : 1.0
29 : 0.9523809523809523
31 : 0.9523809523809523
33 : 0.9047619047619048
35 : 0.9047619047619048
37 : 0.9047619047619048
39 : 0.9047619047619048
41 : 0.9047619047619048
43 : 0.9047619047619048
45 : 0.9047619047619048
47 : 0.9047619047619048
49 : 0.9047619047619048
51 : 0.9047619047619048
53 : 0.9047619047619048
55 : 0.9047619047619048
57 : 0.38095238095238093
59 : 0.3333333333333333
61 : 0.3333333333333333
63 : 0.3333333333333333
65 : 0.3333333333333333
67 : 0.3333333333333333
69 : 0.3333333333333333
71 : 0.3333333333333333
73 : 0.3333333333333333
75 : 0.3333333333333333
77 : 0.3333333333333333
79 : 0.3333333333333333
81 : 0.3333333333333333
83 : 0.3333333333333333


##### Analysis #####

From the results of the accuracy above, we can see two things. The first is that effectively using any value between 3 and 31 results in a very high classifier accuracy on the validation set. We also see that as the setting for $k$ gets higher, the accuracy gets worse. This makes sense since at the very high values, we are essentially finding the majority label of the data points and assigning that label to this new point we need to classify. This is not a particularly useful way of classifying data and doesn't really use any of the actual data point features we might want to use to make the decision.

Since we see that there is a range of points that are possible, I will be selecting the default $k=5$ amount. It is worth noting when $k=[23, 25, 27]$ the accuracy is perfect on the validation set, but we also do not want to over fit our model to that. So I will use the default which still garners a very high accuracy.

Below I will reconstruct the kNN model with the default. If I was interested in being optimal, I likely would have saved the best version above instead of recreating it. For kNNs the "training" step is just collecting the data, but for other machine learning models this sort of optimization would save a lot of time/space/compute.

In [20]:
model = knn(weights='uniform')
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

### Analyze Results ###

Since this is a classification problem, I will examine the accuracy of the model overall, as well as look at the resulting confusion matrix to assess how well the model performs on the different classes.

In [22]:
acc = model.score(X_test, Y_test)
print('Accuracy:', acc)
print()

cmatrix = metrics.confusion_matrix(Y_test, Y_pred)
print('Confusion matrix:')
print(cmatrix)

Accuracy: 1.0

Confusion matrix:
[[18  0  0]
 [ 0 13  0]
 [ 0  0 14]]


This model performs perfectly on the data set, meaning it is very good at separating the 3 iris classes. It is worth noting I have previously applied several machine learning algorithms to this data set that did not perform as perfectly.

| model | accuracy |
| ------- | ------- |
| Naive Bayes | 0.96 |
| SVM -- polynomial | 0.87 |
| SVM -- RBF | 1.0 |
| SVM -- linear | 0.98 |

Thus we find that the kNN model performs better than the Naive Bayes, and several SVM variants.

### Create a Weighted Model ###

As mentioned, kNN can also be implemented to weight the votes of neighbors instead of using majority vote. Thus, points that are further away contribute less to the label decision of a given data point. scikit-learn implements this weighting as inverse the distnace of the neighbor.

Similar to the above example, I will use the validation set to test different $k$ values and then construct the final model.

In [23]:
k_vals = [i for i in range(3, 85, 2)]

for k in k_vals:
    model = knn(n_neighbors=k, weights='distance')
    model.fit(X_train, Y_train)
    acc = model.score(X_val, Y_val)
    print(k, ':', acc)

3 : 0.9523809523809523
5 : 0.9523809523809523
7 : 0.9523809523809523
9 : 0.9523809523809523
11 : 0.9523809523809523
13 : 0.9523809523809523
15 : 0.9523809523809523
17 : 0.9523809523809523
19 : 0.9523809523809523
21 : 0.9523809523809523
23 : 0.9523809523809523
25 : 0.9523809523809523
27 : 0.9523809523809523
29 : 0.9523809523809523
31 : 0.9523809523809523
33 : 0.9523809523809523
35 : 0.9523809523809523
37 : 1.0
39 : 0.9523809523809523
41 : 1.0
43 : 0.9523809523809523
45 : 0.9523809523809523
47 : 0.9523809523809523
49 : 0.9523809523809523
51 : 0.9523809523809523
53 : 0.9523809523809523
55 : 0.9523809523809523
57 : 0.9523809523809523
59 : 0.9523809523809523
61 : 0.9523809523809523
63 : 0.9523809523809523
65 : 0.9523809523809523
67 : 0.9523809523809523
69 : 0.9523809523809523
71 : 0.9523809523809523
73 : 0.9523809523809523
75 : 0.9523809523809523
77 : 0.9523809523809523
79 : 0.9523809523809523
81 : 0.9523809523809523
83 : 0.9523809523809523


In this case, because we use weighting, we see that all values return a high accuracy. This time, the issue of the decreasing accuracy is not present because those points that are further away are weighted much less and so only those that are much closer will be weighted highly and contribute most to the label. Again, for arbitray easiness, I'll construct the model using the default $k=5$. 

In [26]:
model = knn(weights='distance')
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

acc = model.score(X_test, Y_test)
print('Accuracy:', acc)
print()
cmatrix = metrics.confusion_matrix(Y_test, Y_pred)
print(cmatrix)

Accuracy: 1.0

[[18  0  0]
 [ 0 13  0]
 [ 0  0 14]]


##### Analysis #####

In this case, the accuracy is the same as in the uniform kNN case. However, in the case where class distribution is highly skewed, the distance variant can perform much better.